In [94]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image 
import numpy as np

class CoreImages():

    def __init__(self, core_photos, core_x, core_y, slice_window, slice_step=None):
        """
        core_photos: a list of core image paths
        core_x: Tuple (x1,x2) of the x coordinates of the core trays
        core_y: Tuple of tuples ((y1,y2),(y1,y2),(y1,y2),...) with y coordinates for each of the cores within the image
        slice: width to slice each core image into. If not integer divisible by the length of the core, the end of the core will be discarded
        """

        self.slice_window = slice_window
        assert self.slice_window > 0, "Slice window should be greater than 0"
        if slice_step == None:
            self.slice_step = slice_window
        else:
            self.slice_step = slice_step
        assert self.slice_step > 0, "Slice step should be greater than 0"
        

        self.core_photos = core_photos
        self.core_x = core_x
        self.core_y = core_y
    
        self.cores_per_image = len(self.core_y)

        self.core_length = core_x[1] - core_x[0]
        self.slices_per_core = int(np.floor(self.core_length / slice))



    def __getitem__(self, idx):
        return self.core_images[idx]

    def slice_cores(self, core_dir = os.getcwd(), slice_dir = os.getcwd(), verbose = 0):
        """extract the cores from each image, and slice the cores into slices of <slice> width"""
        self.core_slices = []
        core_left, core_right = self.core_x

        for core_photo in self.core_photos:
            if verbose > 0:
                print(f"Processing core photo: {core_photo}")
            for n_core in range(self.cores_per_image):
                #extract the core from the image
                core_photo_img = Image.open(core_photo)
                core_img = core_photo_img.crop((core_left,self.core_y[n_core][0],core_right,self.core_y[n_core][1]))
                core_img.save(f"{core_dir}/{os.path.splitext(os.path.basename(core_photo))[0]}_{n_core}.jpg")

                #slice the core up
                core_width, core_height = core_img.size
                slice_left = 0
                slice_right = self.slice_window
                n_slice = 0
                while slice_right < core_width:
                #for n_slice in range(self.slices_per_core):
                    slice_img = core_img.crop((slice_left,0,slice_right,core_height))
                    slice_img.save(f"{slice_dir}/{os.path.splitext(os.path.basename(core_photo))[0]}_{n_core}_{n_slice}.jpg")
                    slice_img.close()
                    n_slice += 1
                    slice_left += self.slice_step
                    slice_right += self.slice_step

                core_img.close()
                core_photo_img.close()
    

image_dir = "data/Core Images/3-03- 11/"
files = os.listdir(image_dir)
images = [os.path.join(image_dir, f) for f in files]


core_x = 730
core_x_size = 6240
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice = 100
step = slice/2

cores = CoreImages(images, core_x, core_y, slice, step)

cores.slice_cores("data/cores/", "data/slices/", verbose = 1)

Processing core photo: data/Core Images/3-03- 11/S00066841.jpg
Processing core photo: data/Core Images/3-03- 11/S00066846.jpg
Processing core photo: data/Core Images/3-03- 11/S00066844.jpg
Processing core photo: data/Core Images/3-03- 11/S00066847.jpg
Processing core photo: data/Core Images/3-03- 11/S00066845.jpg
Processing core photo: data/Core Images/3-03- 11/S00066848.jpg
Processing core photo: data/Core Images/3-03- 11/S00066842.jpg
Processing core photo: data/Core Images/3-03- 11/S00066843.jpg
